In [25]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

class AdaBoost:
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.alphas = []
        self.models = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        w = np.ones(n_samples) / n_samples

        # Convert y to -1, 1 for AdaBoost internal calculations
        y_converted = np.where(y == 0, -1, 1)

        for _ in range(self.n_estimators):
            model = DecisionTreeClassifier(max_depth=1)
            model.fit(X, y, sample_weight=w) # Fit with original y (0/1)
            predictions = model.predict(X)

            # Convert predictions to -1, 1 for AdaBoost internal calculations
            predictions_converted = np.where(predictions == 0, -1, 1)

            err = np.sum(w * (predictions != y)) / np.sum(w) # Error calculated with original labels

            if err == 0: # If error is 0, the model is perfect. Stop boosting.
                break
            if err >= 1: # Classifier is worse than random; cap error to prevent log of zero or negative
                err = 1 - 1e-10

            alpha = 0.5 * np.log((1.0 - err) / (err + 1e-10)) # Use 1.0 to ensure float division

            self.models.append(model)
            self.alphas.append(alpha)

            # Update weights using converted labels
            w *= np.exp(-alpha * y_converted * predictions_converted)
            w /= np.sum(w)

    def predict(self, X):
        strong_preds = np.zeros(X.shape[0])

        for model, alpha in zip(self.models, self.alphas):
            predictions = model.predict(X)
            # Convert base model predictions to -1, 1 for weighted sum
            predictions_converted = np.where(predictions == 0, -1, 1)
            strong_preds += alpha * predictions_converted

        # Convert final strong prediction back to original 0/1 labels
        return np.where(np.sign(strong_preds) == -1, 0, 1)

if __name__ == "__main__":

    X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    adaboost = AdaBoost(n_estimators=50)
    adaboost.fit(X_train, y_train)

    predictions = adaboost.predict(X_test)

    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    try:
        roc_auc = roc_auc_score(y_test, predictions)
    except ValueError:
        roc_auc = 'Undefined (requires probability scores)'

    print(f"Accuracy: {accuracy * 100}%")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC-AUC: {roc_auc}")

Accuracy: 84.66666666666667%
Precision: 0.8609271523178808
Recall: 0.8387096774193549
F1 Score: 0.8496732026143791
ROC-AUC: 0.8469410456062292
